In [1]:
from keras.models import Model
from keras.applications.resnet50 import ResNet50

from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils

C:\Users\awant\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
resnet_model = ResNet50(weights='imagenet')

In [3]:
input_dim = (224, 224)
image = load_img('Data/dog1.jpg', target_size=input_dim)

In [5]:
type(image)

PIL.Image.Image

In [6]:
image = img_to_array(image)
image = image.reshape((1, *image.shape))
image = preprocess_input(image)

In [8]:
image.shape

(1, 224, 224, 3)

In [9]:
res = resnet_model.predict(image)

In [15]:
res[0][:5]

array([7.6970333e-11, 4.6314010e-09, 5.1344298e-09, 3.8991850e-08,
       5.5097105e-09], dtype=float32)

In [18]:
resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [16]:
import numpy as np

In [23]:
np.argsort(res)[0][-5:]

array([173, 156, 247, 218, 215], dtype=int64)

In [7]:
from keras.applications.vgg16 import VGG16

In [8]:
vgg_model = VGG16(weights='imagenet', include_top=False)

In [9]:
from keras.layers import Dense,GlobalAveragePooling2D

x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
out = Dense(10, activation='softmax')(x)

In [10]:
for layer in vgg_model.layers:
    layer.trainable = False

In [11]:
from keras.optimizers import Adam
opt = Adam()

model = Model(inputs=vgg_model.input, outputs=out)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [12]:
from keras.callbacks import EarlyStopping
callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=0)]

In [13]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [14]:
from keras.utils import np_utils

n_classes = len(np.unique(y_train))
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

In [15]:
n_epochs = 50
batch_size = 512
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.2, verbose=1, callbacks=callbacks)

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 563s 14ms/step - loss: 1.5447 - acc: 0.4683 - val_loss: 1.3404 - val_acc: 0.5299
Epoch 2/50
40000/40000 [==============================] - 572s 14ms/step - loss: 1.2677 - acc: 0.5631 - val_loss: 1.2488 - val_acc: 0.5669
Epoch 3/50
40000/40000 [==============================] - 20691s 517ms/step - loss: 1.1965 - acc: 0.5874 - val_loss: 1.2151 - val_acc: 0.5806
Epoch 4/50
40000/40000 [==============================] - 573s 14ms/step - loss: 1.1532 - acc: 0.6022 - val_loss: 1.2043 - val_acc: 0.5752
Epoch 5/50
40000/40000 [==============================] - 11134s 278ms/step - loss: 1.1163 - acc: 0.6138 - val_loss: 1.1720 - val_acc: 0.5946
Epoch 6/50
40000/40000 [==============================] - 753s 19ms/step - loss: 1.0909 - acc: 0.6226 - val_loss: 1.1812 - val_acc: 0.5861
Epoch 7/50
40000/40000 [==============================] - 470s 12ms/step - loss: 1.0707 - acc: 0.6292 - val_lo